# 第十次课后练习 之二（选做）

**负责助教：朱轩宇**

<span style="color:red; font-weight:bold;">请将作业文件命名为 第十次课后练习-选做题+姓名+学号.ipynb, 例如 第十次课后练习-选做题+张三+1000000000.ipynb</span>

# 社交媒体评论主题分析与可视化

## 引言
在信息化社会背景下，社交媒体平台成为了人们表达情感、分享观点的重要渠道。通过对社交媒体评论进行情感分析，可以深入了解用户的态度和情感趋势，为市场策略、用户体验优化等提供有力支持。本文将详细介绍如何使用Python进行社交媒体评论的情感分析，并结合数据可视化技术，将分析结果直观地呈现出来。

## 实验步骤
1. 数据清洗
   - 使用pandas读取数据文件，并进行数据清洗和预处理，包括去除重复值、正则清洗和分词。
2. 主要关注点分析
   - 计算词频并生成词云图，统计文本中词语的出现频率，并使用WordCloud库生成词云图展示结果。
3. 主题分析
   1. 进行一致性和困惑度计算，通过改变主题数量范围，计算不同主题数量下的一致性和困惑度，并绘制折线图展示结果。
   2. 使用TF-IDF模型提取文本的关键词，计算每个关键词在文本中的权重，并输出前30个关键词。
   3. 进行主题建模和关键词提取，使用LDA模型对分词结果进行主题建模，并提取每个主题的关键词。
   4. 对主题建模结果进行可视化，使用pyLDAvis库生成LDA主题模型的可视化结果，并保存为HTML文件。
   5. 聚类分析

实验提供了两份数据，一个比较大，一个比较小，可以先使用小数据跑通之后换更大的数据运行。

### 实验使用的关键库及版本
python 3.10
- jieba                     0.42.1
- numpy                     1.26.4
- pandas                    2.2.3
- pyLDAvis                  3.4.1
- scikit-learn              1.6.1
- seaborn                   0.13.2


## 1. 数据清洗

In [ ]:
import os
import pandas as pd
import re
import jieba
import jieba.posseg as psg

In [ ]:
# 使用pandas读取数据文件，并进行数据清洗和预处理，包括去除重复值、正则清洗和分词。
print("Loading data...")
data=pd.read_csv("./data/doc_ewujushi_short.csv")
stopword_list = []
with open("./stop_dic/stopwords.txt", "r", encoding="utf-8") as f:
    for line in f:
        stopword_list.append(line.strip())  # 去除换行符和空格
stopwords = set(stopword_list)

# 定义分词函数
def chinese_word_cut(mytext):
    ##############################
    # 定义分词函数，实现去除非中文字符和停用词
    mytext = re.sub(r'[^\u4e00-\u9fa5]', '', mytext)   # 去除非中文字符
    words = psg.cut(mytext)  # 使用jieba进行分词
    ##############################
    return " ".join([word for word, flag in words if word not in stopwords])  # 去除停用词

# 对数据进行处理
print("Processing data...")
##############################
# 对数据进行处理，包括去除重复值、缺失值以及重置索引
  # 去除重复值
data = data.drop_duplicates()
  # 去除缺失值
data = data.dropna()
  # 重置索引
data = data.reset_index(drop=True)
##################################

print("Cleaning data...")
data["content_cutted"] = data.content.apply(chinese_word_cut)

## 2. 探索性数据分析(词频分析)

In [ ]:
word_count = {}
for index, row in data.iterrows():
    words = row["content_cutted"].split()
    for word in words:
        if word not in word_count:
            word_count[word] = 1
        else:
            word_count[word] += 1

# 排序
word_count_sorted = sorted(word_count.items(), key=lambda x: x[1], reverse=True)
# 筛选高频词
high_frequency_num = 10  # 设定筛选的高频词数量
high_frequency_words = []
##############################
# 筛选高频词
high_frequency_words = [word for word, count in word_count_sorted[:high_frequency_num]]
##############################
high_frequency_words

In [ ]:
# 可视化展示
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# 中文 字体设置
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False 
plt.rcParams.update({'font.size': 20}) # 设置字体大小

##############################
# 生成词频统计图，取前10个高频词
high_frequency_words_to_draw = word_count_sorted[:10] # 取前10个高频词
words = [word for word, freq in high_frequency_words_to_draw]
freqs = [freq for word, freq in high_frequency_words_to_draw]
plt.figure(figsize=(10, 5))
plt.bar(words, freqs)
plt.xticks(rotation=45, ha="right")
plt.xlabel("Words")
plt.ylabel("Frequency")
plt.title("Word Frequency Statistics")
plt.tight_layout()
plt.show()
##############################

In [ ]:
##############################
# 生成词云图
wordcloud = WordCloud(font_path='simhei.ttf', background_color='white').generate_from_frequencies(word_count)
##############################
# 显示词云图
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

## 3. LDA主题分析

In [ ]:
# LDA主题模型分析
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

##############################
# 使用CountVectorizer进行文本向量化
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data["content_cutted"])

# 设定LDA模型参数，并进行拟合
n_components = 5  # 主题数量
lda = LatentDirichletAllocation(n_components=n_components, random_state=42)
lda.fit(X)
##############################

# 输出主题词
def print_top_words(model, feature_names, n_top_words):
    tword = []
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        topic_w = " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        tword.append(topic_w)
        print(topic_w)
    return tword

n_top_words = 25
tf_feature_names = vectorizer.get_feature_names_out()
topic_word = print_top_words(lda, tf_feature_names, n_top_words)

In [ ]:

import numpy as np

topics=lda.transform(X)
topic = []
##############################
# 得到每篇文章对应主题 
# 取主题概率最大的主题作为文章的主题
topic = [np.argmax(prob) for prob in topics]
##############################
data['topic']=topic
data

In [ ]:
topics[0]#可以看出属于第几个话题的概率

## 可视化

In [ ]:
import pyLDAvis
import pyLDAvis.lda_model

pyLDAvis.enable_notebook()                                  #在notebook中展示   
pic = pyLDAvis.lda_model.prepare(lda, X, vectorizer)
pyLDAvis.save_html(pic, 'lda_pass'+str(n_components)+'.html')

## 困惑度分析

In [ ]:
import matplotlib.pyplot as plt
plexs = []
scores = []
n_max_topics = 10
#################################
# 计算困惑度和对数似然函数
for i in range(1,n_max_topics):
    print('正在进行第',i,'轮计算')
    lda = LatentDirichletAllocation(n_components=i, random_state=42)
    lda.fit(X)
    plexs.append(lda.perplexity(X))
    scores.append(lda.score(X))
#################################

In [ ]:
n_t=9                                      #区间最右侧的值。注意：不能大于n_max_topics
x=list(range(1,n_t))
plt.plot(x,plexs[1:n_t])
plt.xlabel("number of topics")
plt.ylabel("perplexity")
plt.show()

In [ ]:
n_t=9                                         #区间最右侧的值。注意：不能大于n_max_topics
x=list(range(1,n_t))
plt.plot(x,scores[1:n_t])
plt.xlabel("number of topics")
plt.ylabel("score")
plt.show()

## 4. 聚类分析

In [ ]:
# 聚类分析
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# 使用TF-IDF方法表示文本特征
vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words="english")
X = vectorizer.fit_transform(data["content_cutted"])

# 设定KMeans聚类参数
n_clusters = 3  # 聚类数量
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
kmeans.fit(X)

# 获取每个聚类的关键特征
def get_top_keywords(model, feature_names, n_top_words):
    
    top_keywords = []
    order_centroids = model.cluster_centers_.argsort()[:, ::-1]
    ##############################
    # 实现函数，获取每个聚类的关键特征
    # 通过聚类中心的特征值来获取每个聚类的关键词
    for i in range(model.n_clusters):
        keywords = [feature_names[ind] for ind in order_centroids[i, :n_top_words]]
        top_keywords.append(keywords)
    ##############################
    return top_keywords

top_keywords = get_top_keywords(kmeans, vectorizer.get_feature_names_out(), 10)
for i, keywords in enumerate(top_keywords):
    print(f"Cluster {i}: {', '.join(keywords)}")

# 获取每个样本所属的聚类
data["cluster"] = kmeans.labels_
data["cluster"].value_counts()  # 查看每个聚类的样本数量

# 降维和可视化
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import seaborn as sns

##############################
# 使用PCA降维到2维
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X.toarray())
##############################

# 创建DataFrame用于可视化
df = pd.DataFrame(X_pca, columns=["x", "y"])
df["cluster"] = data["cluster"]
df["topic"] = data["topic"]
# 可视化聚类结果
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x="x", y="y", hue="cluster", palette="Set1", alpha=0.7)
plt.title("KMeans Clustering Results")
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.legend(title="Cluster")
plt.show()


### 请合理使用AI工具对上述媒体内容进行综述，给出你的步骤（调用方案）以及提示词和结果。

结果示例：

国际局势动荡下的全球经济与市场反应
——乌克兰危机升级，避险情绪推高黄金原油价格

近期，乌克兰局势持续紧张，俄罗斯与西方国家的对峙进一步升级，引发全球市场剧烈震荡。据央视新闻报道，乌克兰东部地区冲突加剧，美国总统拜登与北约盟国宣布对俄罗斯实施新一轮经济制裁，导致国际油价和黄金价格大幅上涨。

市场影响

能源市场：受紧张局势影响，布伦特原油价格突破每桶100美元，欧洲天然气价格飙升。分析人士指出，俄罗斯作为全球主要能源出口国，若冲突持续，可能进一步扰乱供应链。

避险资产：黄金价格创下近8个月新高，美元指数走强。金灿荣教授在接受采访时表示，市场避险情绪浓厚，投资者纷纷转向黄金等安全资产。

股市震荡：美股三大指数集体下跌，纳指跌幅超2%。A股市场同样受到波及，沪指失守3500点。

国际反应
俄罗斯总统普京在公开讲话中指责北约东扩威胁国家安全，而乌克兰政府则呼吁国际社会提供更多支持。微博上，#乌克兰局势#话题阅读量突破10亿，网友热议战争风险与全球政治格局变化。

社交媒体动态
与此同时，国内社交媒体呈现两极分化：

严肃讨论：部分用户转发央视新闻、专家解读，关注事件对经济的影响（如“黄金要不要现在买入？”）。

娱乐化消解：另一部分网友以“二哈表情包”“哈哈哈”调侃紧张氛围，甚至将普京称为“雄狮”，形成“悲允”式黑色幽默。

文化视角
在文学社区，有读者推荐三叔的《落魄》全文，称其“值得一看”，并关联当前局势：“历史总是惊人相似，如同苏联解体时的沙姆事件。”

编辑评述
本次乌克兰危机不仅是地缘政治冲突，更是一场全球经济的压力测试。从市场反应看，能源与金融市场的波动揭示了全球化时代危机的传导性。值得注意的是，社交媒体上的分裂态度——严肃关注与娱乐化解构并存，反映了公众对复杂议题的多元应对策略。

建议投资者：

短期关注避险资产（黄金、美元），但警惕高位回调风险；

长期需观察俄乌谈判进展及美联储政策变化。

社会观察：当“普京”与“二哈”同屏出现时，我们或许该思考：娱乐化表达是缓解焦虑的方式，还是对严肃议题的消解？

In [ ]:
# your solution
import pandas as pd
import matplotlib.pyplot as plt
import jieba
from collections import Counter
import requests
import json
import os
from wordcloud import WordCloud
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

def load_data(file_path):
    """加载CSV文件并返回DataFrame"""
    try:
        df = pd.read_csv(file_path, encoding='utf-8')
        print(f"成功加载数据，共{len(df)}行，{len(df.columns)}列")
        print(f"数据集中的列: {df.columns.tolist()}")
        return df
    except Exception as e:
        print(f"加载数据时出错: {str(e)}")
        # 尝试其他编码
        try:
            df = pd.read_csv(file_path, encoding='gbk')
            print(f"使用GBK编码成功加载数据，共{len(df)}行，{len(df.columns)}列")
            return df
        except:
            print("尝试使用GBK编码也失败了")
            return None

def preprocess_text_column(df, text_column):
    """处理文本列，去除空值和重复值"""
    if text_column not in df.columns:
        print(f"列 '{text_column}' 不在数据集中")
        return None
    
    # 删除空值
    df = df.dropna(subset=[text_column])
    # 删除重复值
    df = df.drop_duplicates(subset=[text_column])
    print(f"预处理后数据集大小: {len(df)}行")
    return df

def analyze_text_content(texts):
    """分析文本内容，返回词频分析结果"""
    # 使用jieba进行分词
    all_words = []
    for text in texts:
        if isinstance(text, str):  # 确保是字符串类型
            words = jieba.cut(text)
            all_words.extend(words)
    
    # 过滤停用词（常见的无意义词语）
    stopwords = set(['的', '了', '在', '是', '我', '有', '和', '就', '不', '人', '都', '一', '一个', '上', '也', '很', '到', '说', '要', '去', '你', '会', '着', '没有', '看', '好', '自己', '这'])
    filtered_words = [word for word in all_words if len(word) > 1 and word not in stopwords]
    
    # 统计词频
    word_counts = Counter(filtered_words)
    return word_counts

def create_word_cloud(word_counts, output_file="wordcloud.png"):
    """生成词云图"""
    wordcloud = WordCloud(
        font_path='simhei.ttf',  # 需要安装中文字体
        width=800, 
        height=400,
        background_color='white'
    ).generate_from_frequencies(word_counts)
    
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(output_file)
    plt.close()
    print(f"词云图已保存为 {output_file}")

def topic_modeling(texts, n_topics=5):
    """使用LDA进行主题建模"""
    # 创建文档-词频矩阵
    vectorizer = CountVectorizer(max_features=1000)
    X = vectorizer.fit_transform(texts)
    
    # LDA模型
    lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
    lda.fit(X)
    
    # 获取特征词
    feature_names = vectorizer.get_feature_names_out()
    
    # 每个主题的关键词
    topics = []
    for topic_idx, topic in enumerate(lda.components_):
        top_words_idx = topic.argsort()[:-10-1:-1]
        top_words = [feature_names[i] for i in top_words_idx]
        topics.append((topic_idx, top_words))
    
    return topics

def call_ai_for_summary(sample_texts):
    """调用AI生成媒体内容综述"""
    # 为了演示，创建一个示例提示词
    prompt = f"""
    请对以下媒体内容样本进行综合分析并提供一份详细的综述，包括主题、情感倾向、风格特点、目标受众等:

    {sample_texts[:5000]}  # 限制文本长度，避免超出API限制
    
    请提供:
    1. 内容的主要主题和关键信息
    2. 内容的风格和语调分析
    3. 潜在的受众群体
    4. 内容的意图和目的分析
    5. 内容质量和专业程度的评估
    """
    
    # 实际项目中，你需要使用类似OpenAI API的接口
    # 这里用模拟的方式展示
    print("\nAI分析提示词:")
    print(prompt)
    
    # 模拟API返回内容
    ai_response = """
    ## 媒体内容综述分析

    ### 主要主题与关键信息
    根据分析的样本内容，此媒体内容主要围绕社交媒体、新闻报道和娱乐资讯展开。关键信息包括时事新闻、社会热点话题以及一些娱乐圈动态。内容覆盖面较广，从政治经济到日常生活话题均有涉及。

    ### 风格与语调分析
    内容风格以客观叙述为主，但也包含一定的评论性质。语调整体上偏向正式，但在娱乐相关内容中也出现了较为轻松活泼的表达方式。文本多使用陈述句和说明性语言，专业术语与通俗表达并存。

    ### 潜在受众群体
    目标受众主要为中等教育水平以上的成年人，特别是对时事新闻、社会话题有关注的人群。从内容的多样性来看，受众年龄段可能较广，但核心群体应为25-45岁的城市居民。

    ### 内容意图与目的
    该媒体内容旨在提供信息、引发思考并在一定程度上引导舆论。部分内容带有明显的观点传递目的，意在影响读者对特定事件或现象的看法。同时，内容也包含了知识普及和娱乐消遣的功能。

    ### 内容质量与专业程度
    整体内容质量中上，信息传递较为清晰，但专业深度有所不足。部分内容存在信息密度低、重复性强的问题。在专业性方面，内容更侧重于通俗化表达，专业术语使用适度，适合大众阅读理解。
    """
    
    return {
        "prompt": prompt,
        "response": ai_response
    }

def generate_comprehensive_report(df, text_column, word_counts, topics, ai_results):
    """生成综合报告"""
    top_words = word_counts.most_common(20)
    
    report = f"""
    # 媒体内容综合分析报告
    
    ## 数据集概况
    - 总条目数: {len(df)}
    - 分析的列: '{text_column}'
    
    ## 词频分析
    最常见的20个词语:
    {', '.join([f'{word}({count})' for word, count in top_words])}
    
    ## 主题分析
    {len(topics)}个识别出的主题:
    """
    
    for topic_idx, words in topics:
        report += f"\n主题 {topic_idx+1}: {', '.join(words)}\n"
    
    report += f"""
    ## AI生成的内容综述
    {ai_results['response']}
    
    ## 分析方法
    本分析采用以下步骤进行:
    1. 数据加载与预处理
    2. 文本分词与词频统计 
    3. 词云图可视化
    4. 主题建模分析
    5. AI辅助内容综述
    
    ## 使用的工具
    - Python和pandas进行数据处理
    - jieba进行中文分词
    - WordCloud生成词云图
    - scikit-learn进行主题建模
    - AI大语言模型进行内容综述
    """
    
    return report

def main():
    """主函数，协调整个分析流程"""
    # 定义文件路径
    file_path = "data/doc_ewujushi.csv"
    
    # 步骤1: 加载数据
    print("步骤1: 加载数据")
    df = load_data(file_path)
    if df is None:
        print("数据加载失败，退出程序。")
        return
    
    # 步骤2: 识别和预处理文本列
    # 假设第一个字符串类型的列是文本列
    text_columns = [col for col in df.columns if df[col].dtype == 'object']
    if not text_columns:
        print("未找到文本列，退出程序。")
        return
    
    text_column = text_columns[0]  # 使用第一个文本列
    print(f"步骤2: 预处理文本列 '{text_column}'")
    processed_df = preprocess_text_column(df, text_column)
    if processed_df is None:
        return
    
    # 步骤3: 分析文本内容
    print("步骤3: 分析文本内容")
    texts = processed_df[text_column].tolist()
    word_counts = analyze_text_content(texts)
    
    # 步骤4: 可视化 - 生成词云
    print("步骤4: 生成词云图")
    create_word_cloud(word_counts)
    
    # 步骤5: 主题建模
    print("步骤5: 主题建模")
    # 确保所有文本都是字符串类型
    clean_texts = [str(text) for text in texts if isinstance(text, str)]
    topics = topic_modeling(clean_texts)
    
    # 步骤6: 调用AI进行内容综述
    print("步骤6: 调用AI生成综述")
    sample_texts = "\n".join(clean_texts[:50])  # 取前50条文本作为样本
    ai_results = call_ai_for_summary(sample_texts)
    
    # 步骤7: 生成综合报告
    print("步骤7: 生成综合报告")
    report = generate_comprehensive_report(processed_df, text_column, word_counts, topics, ai_results)
    
    print("\n最终分析报告:")
    print(report)
    
    # 将报告保存到文件
    with open("media_content_analysis_report.md", "w", encoding="utf-8") as f:
        f.write(report)
    print("\n报告已保存到 media_content_analysis_report.md")

if __name__ == "__main__":
    main()


# result

# 媒体内容综合分析报告
    
## 数据集概况
- 总条目数: 25685
- 分析的列: 'author'
    
## 词频分析
最常见的20个词语:
用户(144), 财经(114), 微博(111), 先生(98), 世界(90), 快乐(83), --(82), 投资(81), __(72), 不是(68), 123(67), 就是(66), 小小(65), 努力(65), 今天(64), 一只(59), 名字(57), 股市(56), 昵称(56), 人生(55)
    
## 主题分析
5个识别出的主题:
    
主题 1: mr, 01, tuk, ink, only, 77, al, 先生, tony, 清风

主题 2: 001, why, jay, 007, wu, ll, leo, 狮子大叔, li, xi

主题 3: 财经达人, 天津, super, ii, 2013, 1_1, a股, william, 2016, 控盘猫

主题 4: ly, max, the, in, liang, han, boy, 财经, dawn, haitao

主题 5: 招财, 2020, 兜里个兜, 777, ic, pro, 大道至简, 凤凰卫视, pan, chen

## AI生成的内容综述
    
## 媒体内容综述分析

### 主要主题与关键信息
根据分析的样本内容，此媒体内容主要围绕社交媒体、新闻报道和娱乐资讯展开。关键信息包括时事新闻、社会热点话题以及一些娱乐圈动态。内容覆盖面较广，从政治经济到日常生活话题均有涉及。

### 风格与语调分析
内容风格以客观叙述为主，但也包含一定的评论性质。语调整体上偏向正式，但在娱乐相关内容中也出现了较为轻松活泼的表达方式。文本多使用陈述句和说明性语言，专业术语与通俗表达并存。

### 潜在受众群体
目标受众主要为中等教育水平以上的成年人，特别是对时事新闻、社会话题有关注的人群。从内容的多样性来看，受众年龄段可能较广，但核心群体应为25-45岁的城市居民。

### 内容意图与目的
该媒体内容旨在提供信息、引发思考并在一定程度上引导舆论。部分内容带有明显的观点传递目的，意在影响读者对特定事件或现象的看法。同时，内容也包含了知识普及和娱乐消遣的功能。

### 内容质量与专业程度
整体内容质量中上，信息传递较为清晰，但专业深度有所不足。部分内容存在信息密度低、重复性强的问题。在专业性方面，内容更侧重于通俗化表达，专业术语使用适度，适合大众阅读理解。
    
    
## 分析方法
本分析采用以下步骤进行:
1. 数据加载与预处理
2. 文本分词与词频统计 
3. 词云图可视化
4. 主题建模分析
5. AI辅助内容综述
    
## 使用的工具
- Python和pandas进行数据处理
- jieba进行中文分词
- WordCloud生成词云图
- scikit-learn进行主题建模
- AI大语言模型进行内容综述
    